1) Import libraries and turn CSV file into dataframe and delete rows with empty cells

In [ ]:
import numpy as np
import pandas as pd

# Load data
file_path = '/content/sample_data.csv'
data = pd.read_csv(file_path)

# Drop rows with missing values
data_cleaned = data.dropna()

# Define features and targets
X = data_cleaned[['belief', 'hlow', 'hhigh', 'alpha1', 'alpha2', 'low_herding_share']]
y = data_cleaned[['correlation', 'rmse']].values


1.5) Perform data transformation if needed. In this case, we dublicate the samples with a correlation that is different then the most frequent one.

In [ ]:
# Extract the 'correlation' column
correlation = y[:, 0]

# The two most frequent values provided
frequent_value1 = 0.430170230794728
frequent_value2 = -0.430170231

# Assign weights to samples
weights = np.ones_like(correlation)
weights[(correlation > frequent_value1) & (correlation < frequent_value2)] = 100
weights[(correlation < frequent_value1) & (correlation > frequent_value2)] = 5

# Duplicate the rows based on the weights
X_weighted = np.repeat(X, weights.astype(int), axis=0)
y_weighted = np.repeat(y, weights.astype(int), axis=0)

print(f"Original dataset size: {X.shape[0]}")
print(f"Weighted dataset size: {X_weighted.shape[0]}")


Original dataset size: 7240
Weighted dataset size: 27224


2) Split data into training and test set

In [ ]:
from sklearn.model_selection import train_test_split
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test= train_test_split(
    X_weighted,y_weighted, test_size=0.2, random_state=42)

# Ensure y_train and y_test are NumPy arrays
y_train = np.array(y_train)
y_test = np.array(y_test)

print(f"Training set size: {X_train.shape[0]}")
print(f"Testing set size: {X_test.shape[0]}")

Training set size: 21779
Testing set size: 5445


3) Build a Neural Network and Gradient Boosting Regressor

In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import GradientBoostingRegressor
gbr = GradientBoostingRegressor(random_state=42)
mlp = MLPRegressor(random_state=42)

In [ ]:
import itertools
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neural_network import MLPRegressor

# Generate hidden layer combinations
def generate_hidden_layer_combinations(layers, neurons):
    return [combo for layer in layers for combo in itertools.product(neurons, repeat=layer)]

# Define the number of layers and neurons per layer
num_layers = [2,3,4]
neurons_per_layer = [5,6,7]

# Generate hidden layer combinations
hidden_layers_mlp = generate_hidden_layer_combinations(num_layers, neurons_per_layer)

param_dist_mlp = {
    'hidden_layer_sizes': hidden_layers_mlp,
    'activation': ['relu'],
    'solver': ['lbfgs'],
    'learning_rate': ['adaptive'],
    'learning_rate_init': [0.1],
    'max_iter': [1000, 1500],
}

# Initialize the MLPRegressor
mlp = MLPRegressor()

# Perform RandomizedSearchCV for MLPRegressor
random_search_mlp = RandomizedSearchCV(estimator=mlp, param_distributions=param_dist_mlp, n_iter=300,
                                       cv=5, scoring='neg_mean_squared_error', random_state=42, n_jobs=-1, verbose=0)
random_search_mlp.fit(X_train, y_train)

print("Best hyperparameters for MLPRegressor: ", random_search_mlp.best_params_)


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:307: UserWarning: The total space of parameters 234 is smaller than n_iter=300. Running 234 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best hyperparameters for MLPRegressor:  {'solver': 'lbfgs', 'max_iter': 1500, 'learning_rate_init': 0.1, 'learning_rate': 'adaptive', 'hidden_layer_sizes': (6, 7, 7, 5), 'activation': 'relu'}


In [ ]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingRegressor

# Define the parameter grid for Gradient Boosting Regressor
param_dist_gbr = {
    'estimator__n_estimators': [10, 15, 20],
    'estimator__learning_rate': [0.01, 0.1],
    'estimator__max_depth': [6, 8, 10],
    'estimator__min_samples_split': [2, 3],
    'estimator__min_samples_leaf': [1, 2],
}

# Initialize the Gradient Boosting Regressor
gbr = GradientBoostingRegressor()

# Initialize the MultiOutputRegressor
multi_gbr = MultiOutputRegressor(gbr)

# Perform RandomizedSearchCV for MultiOutputRegressor with Gradient Boosting Regressor
random_search_gbr = RandomizedSearchCV(estimator=multi_gbr, param_distributions=param_dist_gbr, n_iter=300,
                                       cv=5, scoring='neg_mean_squared_error', random_state=42, n_jobs=-1, verbose=1)

# Fit the model with sample weights for the first output
random_search_gbr.fit(X_train, y_train)

# Print the best hyperparameters
print("Best hyperparameters for GradientBoostingRegressor: ", random_search_gbr.best_params_)


Fitting 5 folds for each of 72 candidates, totalling 360 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:307: UserWarning: The total space of parameters 72 is smaller than n_iter=300. Running 72 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best hyperparameters for GradientBoostingRegressor:  {'estimator__n_estimators': 20, 'estimator__min_samples_split': 2, 'estimator__min_samples_leaf': 2, 'estimator__max_depth': 10, 'estimator__learning_rate': 0.1}


4) Evaluate both model on the test set to chose the "winner"

In [ ]:
from sklearn.metrics import mean_squared_error


best_gbr = random_search_gbr.best_estimator_
best_mlp = random_search_mlp.best_estimator_

# Predictions
y_test_pred_gbr = pd.DataFrame(best_gbr.predict(X_test))
y_test_pred_mlp = pd.DataFrame(best_mlp.predict(X_test))

# Calculate MSE for each output
mse_mlp = mean_squared_error(y_test, y_test_pred_mlp)

mse_gbr = mean_squared_error(y_test, y_test_pred_gbr)

# Print individual MSEs
print("GradientBoostingRegressor - MSE Output: ", mse_gbr)

print("MLPRegressor - MSE Output: ", mse_mlp)


GradientBoostingRegressor - MSE Output:  0.028086878026668774
MLPRegressor - MSE Output:  0.045639675394935894


5) Use the newly built surrogate model to generate 10 000 000 parameter combinations, pass them through the surrogate model and find the combination leading to the lowest output

In [ ]:
# Generate new inputs within the specified parameter bounds
def generate_random_params(param_bounds, n_iterations):
    params = {}
    for param, (low, high) in param_bounds.items():
        params[param] = np.random.uniform(low, high, n_iterations)
    return pd.DataFrame(params)

# Define the parameter bounds
param_bounds = {
    'belief': (8, 13),
    'hlow': (0, 0.5),
    'hhigh': (0.5, 1),
    'alpha1': (0, 5),
    'alpha2': (0, 5),
    'low_herding_share': (0, 1)
}

# Generate 10 million random parameter combinations
n_iterations = 10000000
np.random.seed(42)
new_combinations = generate_random_params(param_bounds, n_iterations)

# Use the best model to predict the outputs for the new inputs
best_model = random_search_gbr.best_estimator_
predictions = best_model.predict(new_combinations)

# Add the predictions to the new inputs DataFrame
new_combinations['correlation'] = predictions[:, 0]
new_combinations['rmse'] = predictions[:, 1]

new_combinations['error'] = 0.4*(1-new_combinations['correlation']) + 0.6*new_combinations['rmse']


# Display the 5 combinations that output the highest 'correlation'
top_5_combinations = new_combinations.nsmallest(5, 'error')
print("Top 5 combinations with the highest correlation:")
print(top_5_combinations)

0.3173862


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:458: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:458: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  warnings.warn(


Top 5 combinations with the highest correlation:
            belief      hlow     hhigh    alpha1    alpha2  low_herding_share  \
5818615   8.362252  0.489853  0.578984  0.075125  0.719272           0.228310   
8715712   8.272141  0.489689  0.720229  1.211329  0.025202           0.216874   
7410921   8.226318  0.492748  0.769365  1.344162  0.070500           0.105055   
6579176   9.047509  0.496105  0.607718  1.377633  0.089836           0.197890   
494158   11.779117  0.312089  0.940379  0.047494  0.060807           0.137979   

         correlation      rmse     error  
5818615     0.471023  0.021483  0.224481  
8715712     0.466320  0.022940  0.227236  
7410921     0.447787  0.021179  0.233592  
6579176     0.444062  0.022897  0.236114  
494158      0.435371  0.020474  0.238136  


0.3173862